In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from IPython.display import clear_output

In [2]:
df = pd.read_csv('otomoto_dane.csv')

In [3]:
def wyczysc_dane(df):
    
    path = 'D:/Google Drive/Synchronizowane/Python/Projekty/' \
        'OtoMoto/Otomoto_Dashboard/data/oto_moto_czyste.csv'
    
    war_1 = df.Leasing.isna()
    war_2 = df.Uszkodzony.isna()

    df = df.loc[war_1 & war_2]

    df = df[df.Waluta == 'PLN']
    df = df[~df['Rok produkcji'].isna()]
    df = df[~df['Przebieg'].isna()]
    df = df[~df['Pojemność skokowa'].isna()]
    df = df[~df['Moc'].isna()]

    df['Rok produkcji'] = df['Rok produkcji'].astype(int)

    df.Przebieg = df.Przebieg \
        .str.replace('km','') \
        .str.replace(' ','') \
        .astype(int)

    df['Pojemność skokowa'] = df['Pojemność skokowa'] \
        .str.replace('cm3','') \
        .str.replace(' ','') \
        .astype(int)

    df.Moc = df.Moc \
        .str.replace('KM','') \
        .str.replace(' ','') \
        .astype(int)

    df['Rok ogloszenia'] = df.Data.str[-4:].astype(int)
    df['Wiek'] = df['Rok ogloszenia'] - df['Rok produkcji']


    df.Cena = df.Cena.astype(float)

    cols = ['Marka pojazdu', 'Model pojazdu', 'Rok produkcji', 'Wiek',
           'Przebieg', 'Pojemność skokowa', 'Rodzaj paliwa', 'Moc',
           'Skrzynia biegów','Napęd', 'Typ', 'Cena']
    
    df[cols].to_csv(path, index=False)
    print('Wysłane do data/oto_moto_czyste.csv')

In [4]:
def linki_ze_strony(strona_www):
    
    r = requests.get(strona_www)
    html = BeautifulSoup(r.text, 'html.parser')
    znacz_ofert = html.body.find_all('a', {'class':'offer-title__link'})
    lista_stron = [i.attrs['href'] for i in znacz_ofert]
    
    return lista_stron

def wez_cene(pars_html):
    
    cena = pars_html.body.find('span', {'class':'offer-price__number'})  \
    .find(text=True, recursive=False) \
    .strip() \
    .replace(' ','') \
    .replace(',','.')
    
    waluta = pars_html.body.find('span', {'class':'offer-price__number'}) \
    .find('span', {'offer-price__currency'}) \
    .get_text()
    
    return float(cena), waluta

def oczysc_etykiete(etykieta):
    return etykieta.find('span',{'class':'offer-params__label'}).get_text()
    
def oczysc_wartosc(wartosc):
    return wartosc.find('div',{'class':'offer-params__value'}).get_text().strip()

def wez_parametry(pars_html):
    
    lista = pars_html.body.find('div', {'id':'parameters'}) \
    .find_all('li',{'class':'offer-params__item'})
    
    id = int(pars_html.body.find('span', {'id':'ad_id'}).get_text())
    
    df_temp = pd.DataFrame({'ID':[id]})
    
    for i in lista:
        df_temp[oczysc_etykiete(i)] = oczysc_wartosc(i)
    

    cena, waluta = wez_cene(pars_html=pars_html)
    
    df_temp['Cena'] = cena
    df_temp['Waluta'] = waluta
    df_temp['Data'] = pars_html.find('span', {'class':'offer-meta__value'}).get_text()

    return df_temp

In [7]:
def dodaj_rekord(df_rek):
    
    global df
    df = df.append(df_rek)

        
def czy_jest_ID(df_rek):
    
    global df
    return any(df.ID == df_rek.ID[0])

        
def zbieraj_dane(url, n_strony=1):
        
    while True:
        
        try:
            url_strony = url + str(n_strony)
            linki = linki_ze_strony(url_strony)
        
            for link in linki:
            
                #time.sleep(1)
                r = requests.get(link)
                html_ogl = BeautifulSoup(r.text, 'html.parser')
            
                df_temp = wez_parametry(html_ogl)
            
                # Dodajemy rekord jezeli jest nowy
            
                if czy_jest_ID(df_temp):
                    #n_strony = 0
                    pass
                else:
                    dodaj_rekord(df_temp)
                
                #time.sleep(1)
            
                clear_output(wait=True)
                print(n_strony, df.shape[0])
        except:
            print('Błąd strony')
            time.sleep(60)
    
        n_strony+= 1
        time.sleep(5)
        
        if n_strony == 501:
            break
        

url = 'https://www.otomoto.pl/osobowe/?search%5Border%5D=' \
'created_at_first%3Adesc&search%5Bbrand_program_id%5D%5B0%5D=&search%5Bcountry%5D=&page='
        
#zbieraj_dane(url=url, n_strony=1)
#df['Marka pojazdu'].value_counts()
df.to_csv('otomoto_dane.csv',index_label=False)
wyczysc_dane(df)

Wysłane do data/oto_moto_czyste.csv
